In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
Discrete(3)

Discrete(3)

In [3]:
Box(0,1,shape=(3,3)).sample()

array([[0.40980166, 0.9715955 , 0.91948384],
       [0.9453794 , 0.68243796, 0.471855  ],
       [0.9232006 , 0.5997702 , 0.16871487]], dtype=float32)

In [4]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[157,  54, 100],
       [ 82,  72, 241],
       [  5,  93, 164]])

In [5]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([49.048904], dtype=float32))

In [6]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([14.823278], dtype=float32))])

In [7]:
MultiBinary(4).sample()

array([0, 0, 0, 0], dtype=int8)

In [8]:
MultiDiscrete([5,2,2]).sample()

array([3, 1, 0], dtype=int64)

In [9]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [11]:
env=ShowerEnv()

In [12]:
env.observation_space.sample()

array([91.238365], dtype=float32)

In [13]:
env.reset()

array([39.])

In [14]:
from stable_baselines3.common.env_checker import check_env

In [15]:
check_env(env, warn=True)

In [16]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:4
Episode:2 Score:18
Episode:3 Score:8
Episode:4 Score:-60
Episode:5 Score:-60


In [17]:
env.close()

In [18]:
log_path = os.path.join('Training', 'Logs')

In [19]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
model.learn(total_timesteps=400000)

Logging to Training\Logs\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -21      |
| time/              |          |
|    fps             | 2329     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -28.4        |
| time/                   |              |
|    fps                  | 1480         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0064759646 |
|    clip_fraction        | 0.00674      |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -22.5       |
| time/                   |             |
|    fps                  | 1129        |
|    iterations           | 11          |
|    time_elapsed         | 19          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.007414919 |
|    clip_fraction        | 0.0819      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.961      |
|    explained_variance   | -0.00582    |
|    learning_rate        | 0.0003      |
|    loss                 | 43          |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00707    |
|    value_loss           | 73.9        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -14.7        |
| time/                   |              |
|    fps                  | 1112         |
|    iterations           | 21           |
|    time_elapsed         | 38           |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0042538485 |
|    clip_fraction        | 0.0228       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.571       |
|    explained_variance   | 0.000726     |
|    learning_rate        | 0.0003       |
|    loss                 | 37.3         |
|    n_updates            | 200          |
|    policy_gradient_loss | 0.000629     |
|    value_loss           | 84           |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -11.6        |
| time/                   |              |
|    fps                  | 1076         |
|    iterations           | 31           |
|    time_elapsed         | 58           |
|    total_timesteps      | 63488        |
| train/                  |              |
|    approx_kl            | 0.0017433015 |
|    clip_fraction        | 0.0415       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.535       |
|    explained_variance   | 0.00173      |
|    learning_rate        | 0.0003       |
|    loss                 | 55.7         |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.000258    |
|    value_loss           | 103          |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 4.46         |
| time/                   |              |
|    fps                  | 1063         |
|    iterations           | 41           |
|    time_elapsed         | 78           |
|    total_timesteps      | 83968        |
| train/                  |              |
|    approx_kl            | 0.0017029729 |
|    clip_fraction        | 0.0423       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.399       |
|    explained_variance   | 0.0107       |
|    learning_rate        | 0.0003       |
|    loss                 | 57           |
|    n_updates            | 400          |
|    policy_gradient_loss | -0.00053     |
|    value_loss           | 121          |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60            |
|    ep_rew_mean          | -11.5         |
| time/                   |               |
|    fps                  | 992           |
|    iterations           | 155           |
|    time_elapsed         | 319           |
|    total_timesteps      | 317440        |
| train/                  |               |
|    approx_kl            | 0.00011389589 |
|    clip_fraction        | 0.00674       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0967       |
|    explained_variance   | 0.78          |
|    learning_rate        | 0.0003        |
|    loss                 | 79.7          |
|    n_updates            | 1540          |
|    policy_gradient_loss | 0.000895      |
|    value_loss           | 135           |
-------------------------------------------
-------------------------------------------
| rollout/                |     

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 2.52         |
| time/                   |              |
|    fps                  | 982          |
|    iterations           | 164          |
|    time_elapsed         | 341          |
|    total_timesteps      | 335872       |
| train/                  |              |
|    approx_kl            | 0.0032385483 |
|    clip_fraction        | 0.0155       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.105       |
|    explained_variance   | 0.714        |
|    learning_rate        | 0.0003       |
|    loss                 | 92.5         |
|    n_updates            | 1630         |
|    policy_gradient_loss | 0.00742      |
|    value_loss           | 156          |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -10.7        |
| time/                   |              |
|    fps                  | 977          |
|    iterations           | 173          |
|    time_elapsed         | 362          |
|    total_timesteps      | 354304       |
| train/                  |              |
|    approx_kl            | 0.0010057761 |
|    clip_fraction        | 0.00645      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.114       |
|    explained_variance   | 0.824        |
|    learning_rate        | 0.0003       |
|    loss                 | 43.8         |
|    n_updates            | 1720         |
|    policy_gradient_loss | 0.0013       |
|    value_loss           | 116          |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -8.88       |
| time/                   |             |
|    fps                  | 978         |
|    iterations           | 183         |
|    time_elapsed         | 383         |
|    total_timesteps      | 374784      |
| train/                  |             |
|    approx_kl            | 0.019784912 |
|    clip_fraction        | 0.00933     |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.211      |
|    explained_variance   | 0.676       |
|    learning_rate        | 0.0003      |
|    loss                 | 46.1        |
|    n_updates            | 1820        |
|    policy_gradient_loss | 0.000689    |
|    value_loss           | 142         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60            |
|    ep_rew_mean          | 5.9           |
| time/                   |               |
|    fps                  | 955           |
|    iterations           | 193           |
|    time_elapsed         | 413           |
|    total_timesteps      | 395264        |
| train/                  |               |
|    approx_kl            | 0.00059992017 |
|    clip_fraction        | 0.0142        |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.169        |
|    explained_variance   | 0.752         |
|    learning_rate        | 0.0003        |
|    loss                 | 48.7          |
|    n_updates            | 1920          |
|    policy_gradient_loss | -0.00199      |
|    value_loss           | 111           |
-------------------------------------------
-----------------------------------------
| rollout/                |       

In [21]:
model.save('PPO')

In [23]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

TypeError: render() got an unexpected keyword argument 'mode'